## Part 1: Preprocessing

In [36]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [37]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [38]:
# Create y_df with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [39]:
# Create a list of at least 10 column names to use as X data
select_columns = [
    "Education",
    "Age",
    "DistanceFromHome",
    "JobSatisfaction",
    "OverTime",
    "StockOptionLevel",
    "WorkLifeBalance",
    "YearsAtCompany",
    "YearsSinceLastPromotion",
    "NumCompaniesWorked"
    ]

# Create X_df using your selected columns
X_df = attrition_df[select_columns]

# Show the data types for X_df
X_df.dtypes


,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [44]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)


In [48]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train["OverTime"] = X_train["OverTime"]
X_test['OverTime'] = X_test['OverTime']
X_df['OverTime'].value_counts()

,count
OverTime,
No,1054
Yes,416


In [49]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [51]:
# Create a OneHotEncoder for the Department column
department_ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False, min_frequency=0.2)

# Fit the encoder to the training data
department_ohe.fit(y_train[["Department"]])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department = department_ohe.transform(y_train[["Department"]])
y_test_department = department_ohe.transform(y_test[["Department"]])
y_train_department


array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [52]:
# Create a OneHotEncoder for the Attrition column
attrition_ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False, min_frequency=0.2)

# Fit the encoder to the training data
attrition_ohe.fit(y_train[["Attrition"]])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = attrition_ohe.transform(y_train[["Attrition"]])
y_test_attrition = attrition_ohe.transform(y_test[["Attrition"]])

y_train_attrition


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [55]:
# Find the number of columns in the X training data
num_columns = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_columns,))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation="relu", name="shared1")(input_layer)
shared_layer2 = layers.Dense(128, activation="relu", name="shared2")(shared_layer1)

In [57]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(32, activation="relu", name="department_hidden")(shared_layer2)

# Create the output layer
department_output_layer = layers.Dense(y_train_department.shape[1], activation="softmax", name="department_output")(department_hidden_layer)


In [58]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(32, activation="relu", name="attrition_hidden")(shared_layer2)

# Create the output layer
attrition_output_layer = layers.Dense(y_train_attrition.shape[1], activation="sigmoid", name="attrition_output")(attrition_hidden_layer)

In [75]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer="adam",
              loss={"department_output": "categorical_crossentropy", "attrition_output": "binary_crossentropy"},
              metrics={"department_output": "accuracy", "attrition_output": "accuracy"})

# Summarize the model
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 10)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared1 (Dense)           │ (None, 64)             │            704 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared2 (Dense)           │ (None, 128)            │          8,320 │ shared1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden (Dense) │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden (Dense)  │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ department_hidden[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ attrition_hidden[0][0] │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [76]:
# Train the model
history = model.fit(X_train_scaled,
                    {'department_output': y_train_department,
                     'attrition_output': y_train_attrition},
                    epochs=100,
                    batch_size=32,
                    verbose=1)

Epoch 1/100


35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - attrition_output_accuracy: 0.6767 - department_output_accuracy: 0.6267 - loss: 1.5508
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8253 - department_output_accuracy: 0.6469 - loss: 1.2564
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8268 - department_output_accuracy: 0.6614 - loss: 1.1649
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8283 - department_output_accuracy: 0.6396 - loss: 1.1760
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8457 - department_output_accuracy: 0.6654 - loss: 1.1291
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8528 - department_output_accuracy: 0.6527 - loss: 1.1214
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8592 - department_output_accuracy: 0.6504 - loss: 1.0964
Epoch 8/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 

In [78]:
# Evaluate the model with the testing data
test_data = model.evaluate(X_test_scaled,
                           {'department_output': y_test_department,
                            'attrition_output': y_test_attrition},
                           verbose=2)
test_data

12/12 - 0s - 3ms/step - attrition_output_accuracy: 0.8288 - department_output_accuracy: 0.5462 - loss: 3.9199


[3.919883966445923, 0.8288043737411499, 0.5461956262588501]

In [79]:
# Print the accuracy for both department and attrition
print(f"Department predicions accuracy: {test_data[2]}")
print(f"Attrition predictions accuracy: {test_data[1]}")

Department predicions accuracy: 0.5461956262588501
Attrition predictions accuracy: 0.8288043737411499


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is not the best metric to use on this data. Considering the question is predicting whether employees are likely to leave the company which this data doesn't seem to answer the question. This data shows the accuracy of the model, it needs to identify employees who leave and possibly variables for the reason of their departure of the company, this information would be based on company historical employee turnover.

2. The activation functions that I chose for the output layers were the sigmoid function and the softmax function. The sigmoid function is better suited to determine if an employee will leave or not. The sigmoid function output values are 0 and 1, which are assigned to yes and no results converted into binary numbers. The softmax decision is better suited to determine a more suitable department for an employee, the result is a probability for the departments.

3. A few ways that this model could be improved would be add previous employee data, their reasons for leaving or requesting transfers to another department. Try balancing the data when there is a higher number of employees in certain departments, possibly adding information as to why a department has a higher number employees than another. Possibly do more data splits to give a better understanding of the model's predictions.